<a href="https://colab.research.google.com/github/SeVen4099/Prolog/blob/main/Prolog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Zad. 1

:- discontiguous ojciec/2.
:- discontiguous matka/2.
:- discontiguous malzenstwo/2.
:- discontiguous mezczyzna/1.
:- discontiguous kobieta/1.

% Fakty o rodzicach (1)
ojciec(adam, karol).
ojciec(adam, pawel).
ojciec(adam, anna).
matka(marta, karol).
matka(marta, pawel).
matka(marta, anna).

% Fakty o relacji rodziców (1)
malzenstwo(adam, marta).

% Fakty o płci (1)
mezczyzna(adam).
mezczyzna(karol).
mezczyzna(pawel).
kobieta(marta).
kobieta(anna).

% Fakty o rodzicach (2)
ojciec(krzysztof, daniel).
matka(julia, daniel).

% Fakty o relacji rodziców (2)
malzenstwo(krzysztof, julia).

% Fakty o płci (2)
mezczyzna(krzysztof).
mezczyzna(daniel).
kobieta(julia).

% Fakty o dziadkach
ojciec(jan, marta).
ojciec(jan, julia).
matka(elzbieta, marta).
matka(elzbieta, julia).
mezczyzna(jan).
kobieta(elzbieta).

% Reguła rodzica
rodzic(X, Y) :- ojciec(X, Y).
rodzic(X, Y) :- matka(X, Y).
% Przykładowe zapytanie: ?- rodzic(adam, karol).
% Oczekiwana odpowiedz: true

% Reguła przodka
przodek(X, Y) :- rodzic(X, Y).
% dodaję rekurencję do następnych pokoleń
przodek(X, Y) :-
  rodzic(X, Z),
  przodek(Z, Y).
% Przykładowe zapytanie: ?- przodek(jan, karol).
% Oczekiwana odpowiedz: true

% Reguła malzenstwa
malzenstwo(X, Y) :-
    malzenstwo(Y, X).

% Reguła brata i siostry
brat(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y, % X nie może być bratem samego siebie.
  mezczyzna(X).
% Przykładowe zapytanie: ?- brat(pawel, karol).
% Oczekiwana odpowiedz: true

siostra(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y, % X nie może być siostrą samej siebie.
  kobieta(X).
% Przykładowe zapytanie: ?- siostra(marta, julia).
% Oczekiwana odpowiedz: true

% Reguła rodzeństwa potrzebna do reguły kuzyna
rodzenstwo(X, Y) :-
  brat(X, Y).
rodzenstwo(X, Y) :-
  siostra(X, Y).

% Reguła kuzyna
kuzyn(X, Y) :-
  rodzic(Z, X),
  rodzic(V, Y),
  rodzenstwo(Z, V). % rodzice kuzynów muszą być rodzeństwem
% Przykładowe zapytanie: ?- kuzyn(daniel, karol).
% Oczekiwana odpowiedz: true

Zad. 2

% Fakty o lotniskach
lotnisko(krakow).
lotnisko(manchester).
lotnisko(paryz).
lotnisko(berlin).
lotnisko(lizbona).

% Fakty o połączeniach
polaczenie(krakow, manchester, 250).
polaczenie(manchester, krakow, 250).
polaczenie(krakow, paryz, 400).
polaczenie(paryz, krakow, 400).
polaczenie(krakow, berlin, 500).
polaczenie(berlin, krakow, 500).
polaczenie(manchester, paryz, 700).
polaczenie(paryz, manchester, 700).
polaczenie(manchester, berlin, 900).
polaczenie(berlin, manchester, 900).
polaczenie(manchester, lizbona, 550).
polaczenie(lizbona, manchester, 550).
polaczenie(paryz, lizbona, 600).
polaczenie(lizbona, paryz, 600).

% Reguła sprawdzająca istnienie bezpośredniego połączenia
polaczenie(X, Y) :-
    polaczenie(Y, X, _),
    write('Dostepne polaczenie bezposrednie').
% Przykładowe zapytanie: ?- polaczenie(krakow, berlin).
% Oczekiwana odpowiedz: Dostepne polaczenie bezposrednie


% Pomocnicza reguła wynajdująca trasę bezpośrednią
trasa(X, Y, _, C, [X, Y]) :-
    polaczenie(X, Y, C),
     X \= Y. % Miasto początkowe nie może być miastem docelowym
% Pomocnicza reguła wynajdujaca trasę pośrednią
trasa(X, Y, Odwiedzone, C, [X | ResztaS]) :- % Odwiedzone - lista punktów odwiedzonych już, [X | ResztaS(Reszta drogi)] - lista punktów jakie trzeba odwiedzić.
    polaczenie(X, Z, C1),
    lotnisko(X), lotnisko(Y), % Oba miasta muszą istnieć w bazie
    X \= Y, % Miasto początkowe nie może być miastem docelowym
    \+ member(Z, Odwiedzone), % Pilnuje czy punkt Z nie znajduje się w Odwiedzonych, by zapobiec zapętleniu
    trasa(Z, Y, [Z | Odwiedzone], C2, ResztaS), % Wykorzystanie rekurencji
    C is C1 + C2. % Pełna cena podróży składa się z cen połączeń bezpośrednich

% Reguła sprawdzająca koszt podróży.
koszt_podrozy(X, Y) :-
	lotnisko(X), lotnisko(Y),
    X \= Y,
    findall((C, S), trasa(X, Y, [X], C, S), Wyniki), % Funkcja findall zbiera wszystkie kombinacje cen i tras w liście wyniki. [X] - lista zawierająca punkt startowy, jest początkowym stanem listy Odwiedzone.
    findall(C, member((C, _), Wyniki), Lista), % Funkcja findall zbiera wszystkie ceny wywoływanego połączenia z listy Wyniki i umieszcza je w liście Lista.
	write('Ceny połączenia: '), write(Lista).
% Przykładowe zapytanie: ?- koszt_podrozy(paryz, lizbona).
% Oczekiwana odpowiedz: Ceny połączenia: [600, 1150, 2150, 1300]

% Moja reguła różni się od tej z polecenia, ponieważ pomyślałem, że dobrym pomysłem byłoby wypisanie wszystkich cen, niż żeby program wypisał pierwszą lepszą.
% W razie czego przedstawiam również niżej bardziej zbliżoną wersję do tej z polecenia:
koszt_podrozy1(X, Y, C) :-
    trasa(X, Y, [], C, _), !. % Korzystam z reguły trasa, w której są zawarte wszystkie potrzebne informacje, lecz czas będzie różnić się w zależności od kierunku przejazdu na tej samej trasie.
% Przykładowe zapytanie: ?- koszt_podrozy1(paryz, lizbona, C).
% Oczekiwana odpowiedz: C = 600

% Regula wyznaczająca najkrótszą trasę wg. ceny
najkrotsza_droga(X, Y) :-
    findall((C, S), trasa(X, Y, [X], C, S), Wyniki),
    findall(C, member((C, _), Wyniki), Lista),
    min_list(Lista, MinC), % Wybiera najmniejsze wartości cen z listy.
    member((MinC, MinS), Wyniki), % MinC, MinS znajdują się w liście Wyniki.
    write('Koszt: '), write(MinC), nl, % nl dla nowej lini
    write('Trasa: '), write(MinS), !. % cut
% Przykładowe zapytanie: ?- najkrotsza_droga(berlin, lizbona).
% Oczekiwana odpowiedz: Koszt: 1150, Trasa: [berlin, krakow, manchester, lizbona]

Zad. 3

% Predykaty muszą być dynamiczne by zmieniać ich własności
:- dynamic produkt/3.

% Fakty o produktach (zmienione dane)
produkt(laptop, 100, sektor_a).
produkt(tablet, 75, sektor_a).
produkt(kamera, 40, sektor_b).
produkt(drukarka, 60, sektor_b).
produkt(głośniki, 150, sektor_c).
produkt(mikrofon, 30, sektor_c).

% Reguła dostępności produktu
dostepny_produkt(X) :-
    produkt(X, N, _),
    N > 0, % Jeśli ilość(N) jest większa niż 0, program zwróci wartość True
    write('Status produktu: Dostepny').

% Przykładowe zapytanie: ?- dostepny_produkt(laptop).
% Oczekiwana odpowiedź: Status produktu: Dostepny

% Reguła przenosząca produkt do innego sektora
przenies_produkt(X, Y) :-
    produkt(X, N, OldY),
    retract(produkt(X, N, OldY)), % usuwa
    assertz(produkt(X, N, Y)), % dodaje
    write(produkt(X, N, Y)). % Nowa lokalizacja produktu

% Przykładowe zapytanie: ?- przenies_produkt(laptop, sektor_b).
% Oczekiwana odpowiedź: produkt(laptop, 100, sektor_b)

% Reguła dodająca ilość
uzupelnij_stan(X, N) :-
    produkt(X, N1, Y),
    N2 is N1 + N, % Nowa ilość to suma ilości początkowej i ilości podawanej(N)
    retract(produkt(X, N1, Y)), % usuwa
    assertz(produkt(X, N2, Y)), % dodaje
    write(produkt(X, N2, Y)). % Nowa ilość produktu

% Przykładowe zapytanie: ?- uzupelnij_stan(laptop, 20).
% Oczekiwana odpowiedź: produkt(laptop, 120, sektor_a)

Zad. 4

% Fakty dla miast
miasto(berlin).
miasto(amsterdam).
miasto(paryz).
miasto(madryt).
miasto(rzym).
miasto(lizbona).

% Fakty dla połączeń bezpośrednich
droga(berlin, amsterdam, 3).
droga(amsterdam, berlin, 3).
droga(amsterdam, paryz, 4).
droga(paryz, amsterdam, 4).
droga(paryz, madryt, 5).
droga(madryt, paryz, 5).
droga(rzym, lizbona, 6).
droga(lizbona, rzym, 6).
droga(berlin, paryz, 6).
droga(paryz, berlin, 6).
droga(berlin, madryt, 7).
droga(madryt, berlin, 7).
droga(berlin, lizbona, 8).
droga(lizbona, berlin, 8).
droga(madryt, lizbona, 3).
droga(lizbona, madryt, 3).

% Reguły
droga(X, Y) :-
    droga(X, Y, _),
    write('Istnieje bezposrednie polaczenie: '), write(X), write(' - '), write(Y), !.

% Pomocnicza reguła wynajdująca trasę bezpośrednią
trasa(X, Y, _, T, [X, Y]) :-
    droga(X, Y, T), % Bezpośrednie połączenie
    miasto(X), miasto(Y). % Oba miasta muszą istnieć w bazie

% Pomocnicza reguła wynajdująca trasę pośrednią
trasa(X, Y, Odwiedzone, T, [X | ResztaS]) :- % Odwiedzone - lista punktów odwiedzonych już, [X | ResztaS(Reszta drogi)] - lista punktów jakie trzeba odwiedzić.
    droga(X, Z, T1),
    miasto(X), miasto(Y), % Oba miasta muszą istnieć w bazie
    X \= Y, % Miasto początkowe nie może być miastem docelowym
    \+ member(Z, Odwiedzone), % Pilnuje, czy punkt Z nie znajduje się w Odwiedzonych, by zapobiec zapętleniu
    trasa(Z, Y, [Z | Odwiedzone], T2, ResztaS), % Wykorzystanie rekurencji
    T is T1 + T2. % Pełen czas podróży składa się z czasów indywidualnych połączeń bezpośrednich

% Czas przejazdu dla wszystkich tras
czas_przejazdu(X, Y) :-
    miasto(X), miasto(Y),
    X \= Y,
    findall((T, S), trasa(X, Y, [X], T, S), Wyniki), % Funkcja findall zbiera wszystkie kombinacje czasów i tras w liście Wyniki. [X] - lista zawierająca punkt startowy, jest początkowym stanem listy Odwiedzone.
    findall(T, member((T, _), Wyniki), Lista), % Funkcja findall zbiera wszystkie czasy wywoływanego połączenia z listy Wyniki i umieszcza je w liście Lista.
    write('Czasy przejazdu: '), write(Lista).

% Przykładowe zapytanie: ?- czas_przejazdu(lizbona, amsterdam).
% Oczekiwana odpowiedź: Czasy przejazdu: [10, 12, 11, 12, 7, 9, 14]

% Tak jak w zad.2 moja reguła różni się od tej z polecenia, ponieważ również pomyślałem, że dobrym pomysłem byłoby wypisanie wszystkich czasów, niż żeby program wypisał pierwszy lepszy.
% Wersja bardziej zbliżona do tej z polecenia poniżej:
czas_przejazdu1(X, Y, T) :-
    trasa(X, Y, [], T, _), !. % Korzystam z reguły trasa, w której są zawarte wszystkie potrzebne informacje, lecz czas będzie różnić się w zależności od kierunku przejazdu na tej samej trasie.

% Przykładowe zapytanie: ?- czas_przejazdu1(amsterdam, rom, T).
% Oczekiwana odpowiedź: Czasy przejazdu: T = 10

% Reguła wyznaczająca najkrótszą trasę względem czasu przejazdu
najkrotsza_trasa(X, Y) :-
    findall((T, S), trasa(X, Y, [X], T, S), Wyniki), % Funkcja findall zbiera wszystkie kombinacje czasów i tras w liście Wyniki. [X] - lista zawierająca punkt startowy, jest początkowym stanem listy Odwiedzone.
    findall(T, member((T, _), Wyniki), Lista), % Funkcja findall zbiera wszystkie czasy wywoływanego połączenia z listy Wyniki i umieszcza je w liście Lista.
    min_list(Lista, MinT), % Wybiera najmniejsze wartości czasów z listy.
    member((MinT, MinS), Wyniki), % MinT, MinS znajdują się w liście Wyniki.
    write('Najkrotszy czas przejazdu: '), write(MinT), nl, % dla nowej linijki
    write('Trasa: '), write(MinS), !. % cut

% Przykładowe zapytanie: ?- najkrotsza_trasa(lizbona, paryz).
% Oczekiwana odpowiedź: Najkrótszy czas przejazdu: 9, Trasa: [lizbona, madryt, paryz]

Zad. 5

:- dynamic ocena/3.

% Fakty
ksiazka('Pan Tadeusz', Mickiewicz, powiesc).
ksiazka('Zbrodnia i kara', Dostojewski, powiesc).
ksiazka('Dziady', Mickiewicz, dramat).
ksiazka('Chłopi', Reymont, powiesc).
ksiazka('Kwiaty polskie', Gałczyński, poezja).
ksiazka('Ferdydurke', Gombrowicz, powiesc).
ocena(uzytkownik1, 'Pan Tadeusz', 4).
ocena(uzytkownik1, 'Ferdydurke', 3).
ocena(uzytkownik2, 'Chłopi', 5).
ocena(uzytkownik2, 'Zbrodnia i kara', 4).
preferencje(uzytkownik1, powiesc).
preferencje(uzytkownik2, dramat).

% Dodanie nowej oceny
dodaj_ocene(X, Ksiazka, Ocena) :-
    assertz(ocena(X, Ksiazka, Ocena)), % Dodawanie nowych faktów (ocen).
    write('Pomyslnie dodano: '), write(ocena(X, Ksiazka, Ocena)).
% Przykładowe zapytanie: ?- dodaj_ocene(uzytkownik2, 'Kwiaty polskie', 5).
% Oczekiwana odpowiedz: Pomyslnie dodano: ocena(uzytkownik2, Kwiaty polskie, 5)

% Usunięcie oceny
usun_ocene(X, Ksiazka) :-
    retract(ocena(X, Ksiazka, _)), % Usuwanie faktów (ocen).
    write('Pomyslnie usunieto ocene').
% Przykładowe zapytanie: ?- usun_ocene(uzytkownik2, 'Zbrodnia i kara').
% Oczekiwana odpowiedz: Pomyslnie usunieto ocene

% Reguła polecane_ksiazki/2
polecane_ksiazki(X, Gatunek) :-
    findall(
        Tytul, (ksiazka(Tytul, _, Gatunek), % Funkcja findall zbiera wszystkie tytuły ksiazek,
            preferencje(X, Gatunek),        % których gatunek pokrywa się z preferencjami uzytkownika,
            ocena(X, Tytul, Ocena),         % które zostały ocenione przez uzytkownika
            Ocena >= 3), 	                % na conajmniej 3
        PolecaneKsiazki), 	               % w liście PolecaneKsiazki
    write('Polecane ksiazki: '), write(PolecaneKsiazki).
% Przykładowe zapytanie: ?- polecane_ksiazki(uzytkownik1, powiesc).
% Oczekiwana odpowiedz: Polecane ksiazki: [Pan Tadeusz, Ferdydurke]